In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pandas as pd
import numpy as np

# setting user, api key and access token
plotly.tools.set_credentials_file(username='WendyLiang', api_key='0Mqn1LRKjujs81acpAGt')
mapbox_access_token = 'pk.eyJ1Ijoid2VuZHlsaWFuZyIsImEiOiJjandhZGVzd3cwOHU0NGFuMHBzdHJvNHdhIn0.15zho30YkMBcHvlAVx4J9w'

# load data
df = pd.read_csv('./data/Final_Data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5492 entries, 0 to 5491
Data columns (total 6 columns):
Language                 5492 non-null object
Level of Endangerment    5492 non-null object
Number of Speakers       5309 non-null object
Linguistic Family        5492 non-null object
Countries                5491 non-null object
Coordinates              5492 non-null object
dtypes: object(6)
memory usage: 257.5+ KB


In [2]:
# store rows to be added
new_rows = []
# store index of all the rows that need to be droped
drop_index = []
# iterate through the DataFrame
i = 0
for i in range(len(df)):
    # check if the row have multiple coordinates
    coors = df.iloc[i]['Coordinates'].split(';')
    if len(coors) > 1:
        drop_index.append(i)
        for coor in coors:
            # generate new row to be added to the DataFrame
            new_row = [df.iloc[i]['Language'],
                       df.iloc[i]['Level of Endangerment'],
                       df.iloc[i]['Number of Speakers'],
                       df.iloc[i]['Linguistic Family'],
                       df.iloc[i]['Countries'],
                       coor]
            # store new row
            new_rows.append(new_row)

In [3]:
# drop the rows with multiple coordinates in one cell
df = df.drop(df.index[drop_index])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5301 entries, 1 to 5491
Data columns (total 6 columns):
Language                 5301 non-null object
Level of Endangerment    5301 non-null object
Number of Speakers       5118 non-null object
Linguistic Family        5301 non-null object
Countries                5300 non-null object
Coordinates              5301 non-null object
dtypes: object(6)
memory usage: 289.9+ KB


In [4]:
# append the new rows to the end of original DataFrame
df2 = pd.DataFrame(new_rows, columns=['Language', 'Level of Endangerment', 'Number of Speakers', 'Linguistic Family', 'Countries', 'Coordinates'])
df = df.append(df2, ignore_index = True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 6 columns):
Language                 5822 non-null object
Level of Endangerment    5822 non-null object
Number of Speakers       5639 non-null object
Linguistic Family        5822 non-null object
Countries                5821 non-null object
Coordinates              5822 non-null object
dtypes: object(6)
memory usage: 273.0+ KB


In [5]:
# Separate the Coordinate column into Latitude and Longitude column
lat = []
lon = []
# interate over the DataFrame and store latitude and longitude data in lists
for index, row in df.iterrows():
    coor = row['Coordinates'].split(',')
    if len(coor) == 2:
        lat.append(coor[0])
        lon.append(coor[1])
    else:
        lat.append(None),
        lon.append(None)
        
# turn lists into Series
se_lat = pd.Series(lat)
se_lon = pd.Series(lon)

# add new columns into DataFrame
df['Latitude'] = se_lat.values
df['Longitude'] = se_lon.values

# drop rows with null latitude and longitude
df = df[pd.notnull(df['Latitude'])]
df = df[pd.notnull(df['Countries'])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5763 entries, 0 to 5821
Data columns (total 8 columns):
Language                 5763 non-null object
Level of Endangerment    5763 non-null object
Number of Speakers       5580 non-null object
Linguistic Family        5763 non-null object
Countries                5763 non-null object
Coordinates              5763 non-null object
Latitude                 5763 non-null object
Longitude                5763 non-null object
dtypes: object(8)
memory usage: 405.2+ KB


In [6]:
# export as csv file
# export_csv = df.to_csv ('./data/map_data.csv', index = None, header=True)

Two datasets each have their own scale of endangerment, need to unify them in to UNESCO scale

In [7]:
# change level of endangerment into a same scale (UNESCO scale)
# different level of endangerment in the data
levels = list(df['Level of Endangerment'].value_counts().index)
levels

['Vulnerable',
 'Threatened',
 'Endangered',
 'Definitely endangered',
 'Critically endangered',
 'Severely endangered',
 'Critically Endangered',
 'Severely Endangered',
 'Extinct',
 'At Risk',
 'Dormant',
 'Awakening ()']

The UNESCO scale have five level of endangerment: Vulnerable, Definitely endangered, Severely endangered, Critically endangered, and Extinct.

Classify At Risk and Threatened into Vulnerable; Endangered into Definitely endangered; Severely Endangered into Severely endangered; Critically Endangered into Critically endangered; Dormant into Extinct.

There is only one data in Awakening, so I will leave that.

In [8]:
# unify endangerment scales
df['Level of Endangerment'] = df['Level of Endangerment'].replace(['At Risk', 'Threatened'], 'Vulnerable')
df['Level of Endangerment'] = df['Level of Endangerment'].replace('Endangered', 'Definitely endangered')
df['Level of Endangerment'] = df['Level of Endangerment'].replace('Severely Endangered', 'Severely endangered')
df['Level of Endangerment'] = df['Level of Endangerment'].replace('Critically Endangered', 'Critically endangered')
df['Level of Endangerment'] = df['Level of Endangerment'].replace('Dormant', 'Extinct')
df['Level of Endangerment'] = df['Level of Endangerment'].replace('Awakening ()', 'Awakening')

levels1 = list(df['Level of Endangerment'].value_counts().index)
levels1

['Vulnerable',
 'Definitely endangered',
 'Critically endangered',
 'Severely endangered',
 'Extinct',
 'Awakening']

In [9]:
# export as csv file
export_csv = df.to_csv ('./data/degree_map_data.csv', index = None, header=True)